In [76]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
# used chatGPT

In [77]:
# Load features and labels
features_array = np.load('x_data.npy', allow_pickle=True)
labels_array = np.load('y_label.npy', allow_pickle=True)

In [78]:
columns_to_remove = [
 'font_size', 'right_gaze_point_in_user_x', 'left_gaze_point_in_user_z', 
'left_gaze_point_on_display_area_y', 'right_gaze_point_on_display_area_x', 
'right_gaze_point_in_user_z', 'right_gaze_point_in_user_y', 
 'left_gaze_point_in_user_x', 'right_pupil_diameter', 
 'left_gaze_origin_in_trackbox_z', 'right_gaze_origin_in_trackbox_y', 
 'left_gaze_origin_in_trackbox_x', 'right_gaze_origin_in_user_z', 
 'left_gaze_origin_in_user_x', 'left_gaze_origin_in_user_y', 
 'left_gaze_origin_validity', 'right_gaze_origin_validity', 
 'device_time_stamp', 'left_gaze_point_validity', 
 'right_gaze_point_validity', 'left_pupil_validity', 'right_pupil_validity'
]

In [79]:
processed_dataframes = []
for df in features_array:
  #Drop the columns if they exist, ignore errors if some columns don't exist
  new_df = df.drop(columns=columns_to_remove, errors='ignore')
  processed_dataframes.append(new_df)

features_array = np.array(processed_dataframes, dtype=object)           


In [80]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

lengths = [df.size for df in features_array]
k = int(np.median(lengths))
# Example of handling non-numeric data before flattening
processed_features = []
encoder = OneHotEncoder(sparse=False)  # Initialize one-hot encoder

for df in features_array:
    # Check and transform non-numeric columns if necessary
    for col in df.columns:
        if df[col].dtype == object:
            # Assuming the non-numeric data is categorical and not text like 'Times New Roman'
            transformed = encoder.fit_transform(df[[col]])
            df = pd.concat([df.drop(col, axis=1), pd.DataFrame(transformed)], axis=1)
    
    # Flatten and standardize lengths as before
    flattened = df.values.flatten()
    # Use a fixed length 'k' determined as before
    if len(flattened) > k:
        processed_features.append(flattened[:k])
    else:
        processed_features.append(np.pad(flattened, (0, k - len(flattened)), 'constant'))

X = np.array(processed_features)
y = labels_array

In [81]:
print("NaNs in X:", np.isnan(X).any())
print("Infs in X:", np.isinf(X).any())

# If there are any, you might want to consider replacing them
if np.isnan(X).any() or np.isinf(X).any():
    # Replace NaNs with the mean of the column
    col_mean = np.nanmean(X, axis=0)  # Mean ignoring NaNs
    # Find indices where NaN values are
    inds = np.where(np.isnan(X))
    # Replace NaNs with the mean of each column
    X[inds] = np.take(col_mean, inds[1])

    # Replace infinities with large finite numbers
    X[np.isinf(X)] = 1e+18  # You might choose a suitable finite number

    # Re-check
    print("NaNs in X after replacement:", np.isnan(X).any())
    print("Infs in X after replacement:", np.isinf(X).any())

NaNs in X: True
Infs in X: False
NaNs in X after replacement: False
Infs in X after replacement: False


In [82]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


# Initialize the Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=250,  # Number of trees in the forest
                                  random_state=224182,   # Seed for reproducibility
                                  n_jobs=-1)         # Use all available cores

# Set up stratified cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=224182)

accuracies = []
confusion_matrices = []

# Cross-validation
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Flatten the data
    X_train_flattened = X_train.reshape(X_train.shape[0], -1)
    X_test_flattened = X_test.reshape(X_test.shape[0], -1)
    
    # Train the model on the flattened data
    rf_model.fit(X_train_flattened, y_train)
    
    # Predict on the test set using the flattened test data
    y_pred = rf_model.predict(X_test_flattened)
    
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    
    # Print classification report for each fold
    print(f"Fold Classification Report:\n{classification_report(y_test, y_pred)}")
    
    # Generate confusion matrix for each fold
    cm = confusion_matrix(y_test, y_pred)
    confusion_matrices.append(cm)
    
# Print overall accuracy
print(f"Mean Accuracy: {np.mean(accuracies)}")
print(f"Standard Deviation of Accuracy: {np.std(accuracies)}")




c:\Users\tobia\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Fold Classification Report:
                        precision    recall  f1-score   support

      immersive - easy       0.67      0.40      0.50         5
      immersive - hard       0.50      0.43      0.46         7
    immersive - normal       0.57      0.36      0.44        11
       skimming - easy       0.52      0.46      0.49        56
       skimming - hard       0.64      0.49      0.55        72
     skimming - normal       0.55      0.72      0.62        90
skimming - really hard       0.50      0.60      0.55         5

              accuracy                           0.56       246
             macro avg       0.56      0.49      0.52       246
          weighted avg       0.57      0.56      0.55       246

Fold Classification Report:
                        precision    recall  f1-score   support

      immersive - easy       0.33      0.20      0.25         5
      immersive - hard       1.00      0.43      0.60         7
    immersive - normal       0.25      0.09 

c:\Users\tobia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\tobia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\tobia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold Classification Report:
                         precision    recall  f1-score   support

       immersive - easy       0.40      0.40      0.40         5
       immersive - hard       0.44      0.57      0.50         7
     immersive - normal       0.33      0.09      0.14        11
immersive - really hard       0.00      0.00      0.00         1
        skimming - easy       0.62      0.51      0.56        55
        skimming - hard       0.62      0.50      0.55        72
      skimming - normal       0.54      0.73      0.62        90
 skimming - really hard       1.00      0.60      0.75         5

               accuracy                           0.57       246
              macro avg       0.49      0.43      0.44       246
           weighted avg       0.57      0.57      0.56       246



c:\Users\tobia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\tobia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\tobia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold Classification Report:
                         precision    recall  f1-score   support

       immersive - easy       0.40      0.33      0.36         6
       immersive - hard       1.00      0.43      0.60         7
     immersive - normal       0.62      0.50      0.56        10
immersive - really hard       0.00      0.00      0.00         1
        skimming - easy       0.52      0.44      0.48        55
        skimming - hard       0.56      0.56      0.56        72
      skimming - normal       0.51      0.62      0.56        90
 skimming - really hard       1.00      0.25      0.40         4

               accuracy                           0.53       245
              macro avg       0.58      0.39      0.44       245
           weighted avg       0.55      0.53      0.53       245

Fold Classification Report:
                        precision    recall  f1-score   support

      immersive - easy       0.80      0.67      0.73         6
      immersive - hard       0.7